In [1]:
# 필요 라이브러리 로드
from matplotlib import font_manager, rc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sqlite3,os
import pandas as pd
import platform
from autogluon.tabular import TabularDataset, TabularPredictor
from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
if platform.system() == 'Windows':
# 윈도우인 경우 맑은 고딕 폰트 이용
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf"
                                           ).get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')

#그래프에서 마이너스 기호가 표시되게 하는 설정입니다.
plt.rcParams['axes.unicode_minus'] == False

c:\Users\ban05\anaconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [2]:
df_power = pd.read_csv('../data/한국전력거래소_지역별 시간별 태양광 발전량_20211231.csv', encoding= 'cp949')
df_power.fillna(value = 0, inplace=True)

In [3]:
df_power.columns

Index(['거래일자', '거래시간', '지역', '발전량 '], dtype='object')

In [4]:
df_power.head(2)

,거래일자,거래시간,지역,발전량
0,2018-01-01,1,강원도,13.68
1,2018-01-01,2,강원도,0.00


In [5]:
df_power.drop(columns = ['Unnamed: 0'], inplace =True)

In [6]:
df_power['datetime'] = pd.to_datetime(df_power['datetime'])
df_power['year'] = df_power['datetime'].dt.year
df_power['month'] = df_power['datetime'].dt.month
df_power['day'] = df_power['datetime'].dt.day

In [10]:
df_2 =df_power.groupby(['거래일자','지역'], as_index=False)['발전량 '].mean()
df_2.to_csv('power_production_mean_day_data.csv', encoding='cp949')
df_2

,거래일자,지역,발전량
0,2018-01-01,강원도,20866.921500
1,2018-01-01,경기도,11300.095042
2,2018-01-01,경상남도,25083.820833
3,2018-01-01,경상북도,39324.642625
4,2018-01-01,광주시,4446.730417
...,...,...,...
24832,2021-12-31,전라남도,154515.102833
24833,2021-12-31,전라북도,50341.417042
24834,2021-12-31,제주도,25896.986250
24835,2021-12-31,충청남도,72526.040000


정전 데이터 정제 

In [5]:
df_power = pd.read_csv('processing_data/blackout.csv', encoding= 'cp949')
df_power.fillna(value = 0, inplace=True)

In [6]:
df_power.columns

Index(['Unnamed: 0', '정전피해_지역', '피해_가구수', '시작시간', '종료시간'], dtype='object')

In [7]:
df_power.drop(columns = ['Unnamed: 0'], inplace =True)

In [8]:
df_power['시작시간'] = pd.to_datetime(df_power['시작시간'])
df_power['year'] = df_power['시작시간'].dt.year
df_power['month'] = df_power['시작시간'].dt.month
df_power['day'] = df_power['시작시간'].dt.day

In [9]:
df_2 =df_power.groupby(['year','month' , 'day'], as_index=False)['피해_가구수'].mean()
df_2.to_csv('balckout_mean_day_data.csv', encoding='cp949')
df_2

,year,month,day,피해_가구수
0,2012,6,14,1785.333333
1,2012,6,28,1250.000000
2,2012,6,29,22.000000
3,2012,6,30,103.333333
4,2012,7,18,2277.090909
...,...,...,...,...
145,2022,1,21,2000.000000
146,2022,2,4,2000.000000
147,2022,2,8,2000.000000
148,2022,3,4,12.000000


In [21]:
power_demand_mean_month_2012 = df_2[df_2['year'] == 2012]
power_demand_mean_month_2013 = df_2[df_2['year'] == 2013]
power_demand_mean_month_2014 = df_2[df_2['year'] == 2014]
power_demand_mean_month_2015 = df_2[df_2['year'] == 2015]
power_demand_mean_month_2016 = df_2[df_2['year'] == 2016]
power_demand_mean_month_2017 = df_2[df_2['year'] == 2017]
power_demand_mean_month_2018 = df_2[df_2['year'] == 2018]
power_demand_mean_month_2019 = df_2[df_2['year'] == 2019]
power_demand_mean_month_2020 = df_2[df_2['year'] == 2020]
power_demand_mean_month_2021 = df_2[df_2['year'] == 2021]
power_demand_mean_month_2022 = df_2[df_2['year'] == 2022]

In [22]:
power_demand_mean_month_2012.to_csv('power_demand_mean_month_2012.csv', encoding='cp949')
power_demand_mean_month_2013.to_csv('power_demand_mean_month_2013.csv', encoding='cp949')
power_demand_mean_month_2014.to_csv('power_demand_mean_month_2014.csv', encoding='cp949')
power_demand_mean_month_2015.to_csv('power_demand_mean_month_2015.csv', encoding='cp949')
power_demand_mean_month_2016.to_csv('power_demand_mean_month_2016.csv', encoding='cp949')
power_demand_mean_month_2017.to_csv('power_demand_mean_month_2017.csv', encoding='cp949')
power_demand_mean_month_2018.to_csv('power_demand_mean_month_2018.csv', encoding='cp949')
power_demand_mean_month_2019.to_csv('power_demand_mean_month_2019.csv', encoding='cp949')
power_demand_mean_month_2020.to_csv('power_demand_mean_month_2020.csv', encoding='cp949')
power_demand_mean_month_2021.to_csv('power_demand_mean_month_2021.csv', encoding='cp949')
power_demand_mean_month_2022.to_csv('power_demand_mean_month_2022.csv', encoding='cp949')